In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

from importlib import reload

import models
import plotting
import dataloaders as dl
import traintest as tt

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
import resnet
rs50 = resnet.ResNet50().to(device)

In [ ]:
lr = .1

optimizer = optim.Adam( rs50.parameters(), lr=lr, weight_decay=0.0001)
for epoch in range(50):
    tt.train(rs50, device, dl.CIFAR10_train_loader, optimizer, epoch)

optimizer.param_groups[0]['lr'] *= .1
for epoch in range(50):
    tt.train(rs50, device, dl.CIFAR10_train_loader, optimizer, epoch)

optimizer.param_groups[0]['lr'] *= .1
for epoch in range(50):
    tt.train(rs50, device, dl.CIFAR10_train_loader, optimizer, epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.396156
Train Epoch: 0 [10000/50000 (20%)]	Loss: 2.448792
Train Epoch: 0 [20000/50000 (40%)]	Loss: 1.931553
Train Epoch: 0 [30000/50000 (60%)]	Loss: 1.911191
Train Epoch: 0 [40000/50000 (80%)]	Loss: 1.931136
Train Epoch: 1 [0/50000 (0%)]	Loss: 1.941912
Train Epoch: 1 [10000/50000 (20%)]	Loss: 1.745904
Train Epoch: 1 [20000/50000 (40%)]	Loss: 1.660045
Train Epoch: 1 [30000/50000 (60%)]	Loss: 1.622159
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1.579715
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.711847
Train Epoch: 2 [10000/50000 (20%)]	Loss: 1.660302


In [ ]:
torch.save(rs50, 'SavedModels/resnet50.pth')

In [ ]:
rs50 = torch.load('SavedModels/resnet50.pth')
pca_metric = models.PCAMetric(dl.X_CIFAR10, min_sv_factor=1000.)
gmm = models.GMM(1000, 3072, metric=pca_metric)
gmm.find_solution(dl.X_CIFAR10, initialize=True, iterate=False, use_kmeans=False)
gmm_model = models.RobustModel(rs50, gmm, -7., dim=3072).to(device)
torch.save(gmm_model, 'SavedModels/gmm_CIFAR10.pth')

In [ ]:
gmm_model = torch.load('SavedModels/gmm_CIFAR10.pth')
#rs50 = torch.load('SavedModels/resnet50.pth')

In [ ]:
gmm_model.mm.logvar.data -= 3

In [ ]:
gmm_model.loglam.data = torch.tensor(-7., device=device)

data = enumerate(dl.CIFAR10_test_loader).__next__()[1][0]
y = gmm_model.base_model(data.to(device))
y2 = gmm_model(data.to(device))

plotting.plot_samples([y, y2], data, dataset = 'CIFAR10')

In [ ]:
data = enumerate(dl.CIFAR100_test_loader).__next__()[1][0]
y = gmm_model.base_model(data.to(device))
y2 = gmm_model(data.to(device))

plotting.plot_samples([y, y2], data, dataset = 'CIFAR10')

In [ ]:
data = torch.rand(10,3,32,32)
y = gmm_model.base_model(data.to(device))
y2 = gmm_model(data.to(device))

plotting.plot_samples([y, y2], data, dataset = 'CIFAR10')

In [ ]:
min_conf = .1
tt.test(gmm_model, device, dl.CIFAR10_test_loader, min_conf=min_conf)
tt.test(gmm_model.base_model, device, dl.CIFAR10_test_loader, min_conf=min_conf)

tt.test_adv(gmm_model, device, dl.CIFAR100_test_loader, min_conf=min_conf)
tt.test_adv(gmm_model.base_model, device, dl.CIFAR100_test_loader, min_conf=min_conf)

In [ ]:
plt.imshow(gmm_model.mm.metric.comp_vecs[0,:].data.cpu().view(3,32,32).transpose(0,2)[:,:,1].numpy())

In [ ]:
plt.plot(gmm_model.mm.metric.singular_values.data.cpu().numpy()[200:])